In [1]:
import pyppeteer

In [2]:
import asyncio
from pyppeteer import launch
from bs4 import BeautifulSoup
import markdownify
import time

In [3]:
async def getcontentJSON(each):
    ht_c = await(await each.getProperty("innerHTML")).jsonValue()
    soup = BeautifulSoup(ht_c,"html.parser")
    link = soup.find("h4").find("a")['href']
    return link

In [4]:
async def main1(pageno,startlink):
    global all_links
    browser = await launch(headless=False)
    page = await browser.newPage()
    await page.goto(startlink,{'waitUntil':"domcontentloaded"})
    for _ in range(pageno):
        cb = await page.querySelector("div.contentBox")
        fnbp =await cb.querySelector("div.newsBoxPost")
        list_nbp = await fnbp.querySelectorAll("div.newsBoxPost")
        all_links.extend([await getcontentJSON(each) for each in list_nbp])
        nxtlink = await(await (await page.querySelector("a.nextpostslink")).getProperty("href")).jsonValue()
        print(nxtlink)
        await asyncio.gather(
            page.waitForNavigation(),
            page.goto(nxtlink,{'waitUntil':"domcontentloaded"})
        )
    await browser.close()

In [7]:
all_links=[]
await main1(5,'https://taxguru.in/filters/?cats=rbi&type=articles&court=0&pyear=0&pmonth=&filters=Y')

https://taxguru.in/filters/page/2/?cats=rbi&type=articles&court=0&pyear=0&pmonth&filters=Y
https://taxguru.in/filters/page/3/?cats=rbi&type=articles&court=0&pyear=0&pmonth&filters=Y
https://taxguru.in/filters/page/4/?cats=rbi&type=articles&court=0&pyear=0&pmonth&filters=Y
https://taxguru.in/filters/page/5/?cats=rbi&type=articles&court=0&pyear=0&pmonth&filters=Y
https://taxguru.in/filters/page/6/?cats=rbi&type=articles&court=0&pyear=0&pmonth&filters=Y


In [8]:
all_links=list(set(all_links))

In [9]:
all_links

['https://taxguru.in/rbi/framework-compromise-settlements-technical-write-offs.html',
 'https://taxguru.in/rbi/rbi-guidelines-banks-nbfcs-penal-charges.html',
 'https://taxguru.in/rbi/risk-based-internal-audit-mechanism-banks-change-compliance-procedures-substantial.html',
 'https://taxguru.in/rbi/impact-increasing-repo-rates-borrowers-managing-financial-burden.html',
 'https://taxguru.in/rbi/understanding-fema-regulations-nris-residents-lending.html',
 'https://taxguru.in/rbi/external-commercial-borrowings-regulations-eligibility-reporting.html',
 'https://taxguru.in/rbi/31-faqs-filling-foreign-liability-asset-fla-return-india.html',
 'https://taxguru.in/rbi/overseas-investment-rules-2022-resident-individuals-indian-entities.html',
 'https://taxguru.in/rbi/sovereign-gold-bonds-features-advantages-income-tax-gst.html',
 'https://taxguru.in/rbi/decoding-provisions-filing-fc-gpr-vide-fema-guidelines.html',
 'https://taxguru.in/rbi/rbi-investment-portfolio-guidelines-2023-key-updates.html

In [10]:
async def getcontentpage(page,eachlink):
    mdict={"link":eachlink}
    await page.goto(eachlink,{'waitUntil':"domcontentloaded"})
    title = await page.querySelector("div.homeTitle")
    title = await(await title.getProperty("innerHTML")).jsonValue()
    soup = BeautifulSoup(title,"html.parser")
    mdict["title"] = soup.find("a")["title"]
    content = await page.querySelector("div.fsize16")
    content = await(await content.getProperty("innerHTML")).jsonValue()
    soup = BeautifulSoup(content,"html.parser")
    soup.script.decompose()
    mdict["content"] = soup.get_text()
    table = []
    table_c = []
    for i in soup.find_all("table"):
        table.append(markdownify.markdownify(str(i), heading_style="ATX"))
        table_c.append(str(i))
    mdict["table"] = table
    mdict["table_c"] = table_c
    return mdict

In [11]:
async def main2(all_links):
    browser = await launch(headless=False)
    page = await browser.newPage()
    data= [await getcontentpage(page,link) for link in all_links]
    await browser.close()
    return data

In [13]:
fdata = await main2(all_links[:10])

In [14]:
fdata

[{'link': 'https://taxguru.in/rbi/framework-compromise-settlements-technical-write-offs.html',
  'title': 'Framework for Compromise Settlements and Technical Write-offs',
  'content': '\n\n\n\n\n\n\nReserve Bank of India issued their circular DOR.STR.REC.20/21.04.048/2023-24 dated June 08, 2023 addressed to all Regulated Entities (RE) coming under RBI on Framework for Compromise Settlements and Technical Write-offs.\nThe preamble of RBI circular states, “The Reserve Bank of India has issued various instructions to regulated entities (REs) regarding compromise settlements in respect of stressed accounts from time to time, including the Prudential Framework for Resolution of Stressed Assets dated June 7, 2019 (“Prudential Framework”), which recognises compromise settlements as a valid resolution plan. With a view to provide further impetus to resolution of stressed assets in the system as well as to rationalise and harmonise the instructions across all REs, as announced in the Statement 

In [15]:
import json
with open("finaldata.json","w") as f:
    json.dump(fdata,f)